In [1]:
ls

sample_data/


In [2]:
ls Features/

ls: cannot access 'Features/': No such file or directory


In [3]:
ls

sample_data/


In [87]:
data=np.load("/content/Dataset_shuai/Cluster_index_mat/s_10_cluster_index.npy",allow_pickle=True)

In [37]:

len(os.listdir("/content/Dataset_shuai/Features"))

50

In [92]:
data.item()['cluster_index_mat'].shape

(45, 54, 45)

In [75]:
import scipy.io as sc
data=sc.loadmat("/content/Dataset_shuai/label.mat")

In [84]:
data['label'].dtype

dtype('uint8')

In [70]:
#feature
data.item()['feature_mat'].shape

(400, 1632)

In [86]:
import numpy as np
data=np.load("s_100_cluster_index.npy",allow_pickle=True)

FileNotFoundError: [Errno 2] No such file or directory: 's_100_cluster_index.npy'

In [5]:
!lscpu

Architecture:                x86_64
  CPU op-mode(s):            32-bit, 64-bit
  Address sizes:             46 bits physical, 48 bits virtual
  Byte Order:                Little Endian
CPU(s):                      2
  On-line CPU(s) list:       0,1
Vendor ID:                   GenuineIntel
  Model name:                Intel(R) Xeon(R) CPU @ 2.20GHz
    CPU family:              6
    Model:                   79
    Thread(s) per core:      2
    Core(s) per socket:      1
    Socket(s):               1
    Stepping:                0
    BogoMIPS:                4399.99
    Flags:                   fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pg
                             e mca cmov pat pse36 clflush mmx fxsr sse sse2 ss h
                             t syscall nx pdpe1gb rdtscp lm constant_tsc rep_goo
                             d nopl xtopology nonstop_tsc cpuid tsc_known_freq p
                             ni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2ap
                   

In [39]:
data.item()['feature_mat'].shape

(400, 1632)

In [4]:
!gdown 1_X4mnUK1KtlFiUIpCwZlGovQBPqLlYJ1

Downloading...
From (original): https://drive.google.com/uc?id=1_X4mnUK1KtlFiUIpCwZlGovQBPqLlYJ1
From (redirected): https://drive.google.com/uc?id=1_X4mnUK1KtlFiUIpCwZlGovQBPqLlYJ1&confirm=t&uuid=2e6955d1-a5f0-484d-b207-b219e2154300
To: /content/Dataset_shuai_zip.zip
100% 2.51G/2.51G [00:34<00:00, 72.6MB/s]


In [7]:
!unzip Dataset_shuai_zip.zip

Archive:  Dataset_shuai_zip.zip
   creating: Dataset_shuai_zip/
   creating: Dataset_shuai_zip/Cluster_index_mat/
  inflating: Dataset_shuai_zip/Cluster_index_mat/s_88_cluster_index.npy  
  inflating: Dataset_shuai_zip/Cluster_index_mat/s_89_cluster_index.npy  
  inflating: Dataset_shuai_zip/Cluster_index_mat/s_8_cluster_index.npy  
  inflating: Dataset_shuai_zip/Cluster_index_mat/s_90_cluster_index.npy  
  inflating: Dataset_shuai_zip/Cluster_index_mat/s_91_cluster_index.npy  
  inflating: Dataset_shuai_zip/Cluster_index_mat/s_92_cluster_index.npy  
  inflating: Dataset_shuai_zip/Cluster_index_mat/s_93_cluster_index.npy  
  inflating: Dataset_shuai_zip/Cluster_index_mat/s_94_cluster_index.npy  
  inflating: Dataset_shuai_zip/Cluster_index_mat/s_95_cluster_index.npy  
  inflating: Dataset_shuai_zip/Cluster_index_mat/s_96_cluster_index.npy  
  inflating: Dataset_shuai_zip/Cluster_index_mat/s_97_cluster_index.npy  
  inflating: Dataset_shuai_zip/Cluster_index_mat/s_98_cluster_index.npy  

In [12]:
ls

Cluster_index_mat/  Features/  label.mat*


In [ ]:
/content/Dataset_shuai_zip/Cluster_index_mat/
/content/Dataset_shuai_zip/Features/data
/content/Dataset_shuai_zip/label.mat

In [17]:
# brain_dataset.py
"""
Custom PyTorch Dataset and DataLoaders for the neuroimaging toy dataset.
Assumptions:
- label.mat contains 'label' variable (shape: 500 or 500×1)
- Feature files: s1_feature.mat, s2_feature.mat, ..., s500_feature.mat
  Inside: 'F' → (400, 1632) float array
- Cluster files: s1_cluster_index.mat, s2_cluster_index.mat, ..., s500_cluster_index.mat
  Inside: 'C' → (45, 54, 45) integer array
"""

import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import scipy.io as sio
from sklearn.model_selection import train_test_split
from typing import Optional, Dict, Any
# import train_config


class BrainROIDataset(Dataset):
    """
    Loads per-subject ROI feature matrices (400×1632) and optionally cluster maps.
    Labels are loaded once from label.mat and indexed by subject number (1-based).
    """
    def __init__(
        self,
        subject_ids: list[int],
        label_data_dir: str,
        feature_data_dir: str,
        cluster_data_dir: str,
        load_cluster: bool = False,
        feature_key: str = 'F',
        cluster_key: str = 'C'
    ):
        """
        Args:
            subject_ids: List of subject IDs (integers 1 to 500)
            data_dir: Root directory containing all .mat files
            load_cluster: Whether to load the 3D cluster maps (memory intensive)
            feature_key: Key name for feature matrix in .mat file
            cluster_key: Key name for cluster index in .mat file
        """
        self.subject_ids = sorted(subject_ids)
        self.data_dir = label_data_dir
        self.feature_data=feature_data_dir
        self.cluster_data=cluster_data_dir
        self.load_cluster = load_cluster
        self.feature_key = feature_key
        self.cluster_key = cluster_key

        # Load global labels once
        label_path = self.data_dir
        if not os.path.exists(label_path):
            raise FileNotFoundError(f"Label file not found: {label_path}")

        labels_mat = sio.loadmat(label_path)
        self.labels = labels_mat['label'].flatten().astype(np.int64)  # shape (500,)

        if len(self.labels) != 500:
            raise ValueError(f"Expected 500 labels, got {len(self.labels)}")

    def __len__(self) -> int:
        return len(self.subject_ids)

    def __getitem__(self, idx: int) -> Dict[str, Any]:
        sid = self.subject_ids[idx]  # 1-based subject ID

        # 1. Load feature matrix
        feature_path = os.path.join(self.feature_data, f"s_{sid}_feature.npy")
        if not os.path.exists(feature_path):
            raise FileNotFoundError(f"Feature file missing: {feature_path}")

        feat_data = np.load(feature_path,allow_pickle=True)
        features=feat_data.item()['feature_mat']
        # features = feat_data[self.feature_key].astype(np.float32)  # (400, 1632)
        
        cluster_path=os.path.join(self.cluster_data, f"s_{sid}_cluster_index.npy")
        if not os.path.exists(cluster_path):
            raise FileNotFoundError(f"Cluster file missing: {cluster_path}")
        clus_data=np.load(cluster_path, allow_pickle=True)
        clusters_matrix=clus_data.item()['cluster_index_mat']

        # 2. Optional: Load cluster map
        # cluster = None
        # if self.load_cluster:
        #     cluster_path = os.path.join(self.data_dir, f"s{sid}_cluster_index.mat")
        #     if not os.path.exists(cluster_path):
        #         raise FileNotFoundError(f"Cluster file missing: {cluster_path}")

        #     clust_data = sio.loadmat(cluster_path)
        #     cluster = clust_data[self.cluster_key].astype(np.int16)  # (45,54,45)

        # 3. Get corresponding label (0-based index in array)
        label = self.labels[sid - 1]

        # Prepare output
        item = {
            'features': torch.from_numpy(features),     # [400, 1632]
            'label': torch.tensor(label, dtype=torch.long),
            'subject_id': sid
        }

        # if cluster is not None:
        #     item['cluster_map'] = torch.from_numpy(cluster)

        return item


def create_dataloaders(
    label_data_dir,
    feature_data_dir,
    cluster_data_dir,
    batch_size: int = 16,
    num_workers: int = 4,
    pin_memory: bool = True,
    load_cluster: bool = False,
    val_size: float = 0.15,
    test_size: float = 0.15,
    random_state: int = 42
) -> tuple[DataLoader, DataLoader, DataLoader]:
    """
    Creates train/val/test DataLoaders with standard split (70/15/15 by default)

    Returns:
        (train_loader, val_loader, test_loader)
    """
    all_subjects = list(range(1, 501))  # 1 to 500

    # Stratified split (though binary labels, still good practice)
    train_subjs, temp_subjs = train_test_split(
        all_subjects,
        test_size=(val_size + test_size),
        random_state=random_state
    )

    val_subjs, test_subjs = train_test_split(
        temp_subjs,
        test_size=test_size / (val_size + test_size),
        random_state=random_state
    )

    print(f"Dataset split → Train: {len(train_subjs)} | Val: {len(val_subjs)} | Test: {len(test_subjs)}")

    train_ds = BrainROIDataset(train_subjs, label_data_dir, feature_data_dir, cluster_data_dir, load_cluster=load_cluster)
    val_ds   = BrainROIDataset(val_subjs,   label_data_dir, feature_data_dir, cluster_data_dir, load_cluster=load_cluster)
    test_ds  = BrainROIDataset(test_subjs,  label_data_dir, feature_data_dir, cluster_data_dir, load_cluster=load_cluster)

    train_loader = DataLoader(
        train_ds,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=pin_memory,
        drop_last=True
    )

    val_loader = DataLoader(
        val_ds,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=pin_memory
    )

    test_loader = DataLoader(
        test_ds,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=pin_memory
    )

    return train_loader, val_loader, test_loader


# ──────────────────────────────────────────────────────────────────────────────
# Quick test / example usage
# ──────────────────────────────────────────────────────────────────────────────



In [ ]:
/content/Dataset_shuai/Cluster_index_mat/
/content/Dataset_shuai/Features/
/content/Dataset_shuai/label.mat

In [46]:
ls

Cluster_index_mat/  Features/  label.mat


In [18]:

# Change this to your actual data location
DATA_ROOT = "/content/Dataset_shuai/"   # ← IMPORTANT: update this path

train_loader, val_loader, test_loader = create_dataloaders(
    label_data_dir="/content/Dataset_shuai_zip/label.mat",
    feature_data_dir="/content/Dataset_shuai_zip/Features",
    cluster_data_dir="/content/Dataset_shuai_zip/Cluster_index_mat/",
    batch_size=8,
    load_cluster=False,       # ← Set True only if you really need it
)

# Get one batch to verify
batch = next(iter(train_loader))

print("\nBatch shapes:")
print(f"features    : {batch['features'].shape}")     # [batch, 400, 1632]
print(f"labels      : {batch['label'].shape}")        # [batch]
print(f"subject_ids : {batch['subject_id']}")

if 'cluster_map' in batch:
    print(f"cluster_map : {batch['cluster_map'].shape}")

    

Dataset split → Train: 350 | Val: 75 | Test: 75


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)



Batch shapes:
features    : torch.Size([8, 400, 1632])
labels      : torch.Size([8])
subject_ids : tensor([134, 145, 345, 453, 326, 200, 251, 229])


In [20]:
batch['label']

tensor([2, 2, 1, 2, 1, 2, 2, 1])

In [29]:
np.load("/content/Dataset_shuai/Features/s_257_feature.npy")

FileNotFoundError: [Errno 2] No such file or directory: '/content/Dataset_shuai/Features/s_257_feature.npy'

In [6]:
from google.colab import drive
drive.mount('/content/drive')

ValueError: mount failed

In [9]:
!wget -O "Dataset_shuai" "https://drive.google.com/drive/u/0/folders/1FqNaf3EHFwARjTg6d14OZBzM3h-2Rtts"

--2026-01-10 06:22:37--  https://drive.google.com/drive/u/0/folders/1FqNaf3EHFwARjTg6d14OZBzM3h-2Rtts
Resolving drive.google.com (drive.google.com)... 173.194.206.113, 173.194.206.100, 173.194.206.138, ...
Connecting to drive.google.com (drive.google.com)|173.194.206.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://drive.google.com/drive/folders/1FqNaf3EHFwARjTg6d14OZBzM3h-2Rtts [following]
--2026-01-10 06:22:37--  https://drive.google.com/drive/folders/1FqNaf3EHFwARjTg6d14OZBzM3h-2Rtts
Reusing existing connection to drive.google.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://accounts.google.com/ServiceLogin?service=wise&passive=1209600&osid=1&continue=https://drive.google.com/drive/folders/1FqNaf3EHFwARjTg6d14OZBzM3h-2Rtts&followup=https://drive.google.com/drive/folders/1FqNaf3EHFwARjTg6d14OZBzM3h-2Rtts [following]
--2026-01-10 06:22:37--  https://accounts.google.com/ServiceLogin?service=wise&passive=1209600&osid=

In [ ]:
!gdown --folder --remaining-ok 1FqNaf3EHFwARjTg6d14OZBzM3h-2Rtts

In [35]:
import os

In [32]:
!rm Dataset_shuai

In [27]:
os.listdir('/content/Dataset_shuai')

NotADirectoryError: [Errno 20] Not a directory: '/content/Dataset_shuai'

In [51]:
len(os.listdir('Dataset_shuai/Features'))

50

In [43]:
ls

Dataset_shuai/  sample_data/


In [44]:
!rm -r Dataset_shuai/

In [45]:
ls

sample_data/


In [52]:
!gdown 1zubFt72nJLNZGgKMxgNsGThVWH3fYbmT

Downloading...
From (original): https://drive.google.com/uc?id=1zubFt72nJLNZGgKMxgNsGThVWH3fYbmT
From (redirected): https://drive.google.com/uc?id=1zubFt72nJLNZGgKMxgNsGThVWH3fYbmT&confirm=t&uuid=453a47c6-8e1f-49ff-858b-204a53f45a86
To: /content/toy_data.zip
100% 2.52G/2.52G [00:58<00:00, 43.4MB/s]


In [53]:
ls

Dataset_shuai/  sample_data/  toy_data.zip


In [54]:
!unzip toy_data.zip

Archive:  toy_data.zip
mapname:  conversion of  failed
 extracting: label.mat               
 extracting: s_1_feature.mat         
 extracting: s_4_feature.mat         
 extracting: s_6_feature.mat         
 extracting: s_7_feature.mat         
 extracting: s_2_feature.mat         
 extracting: s_5_feature.mat         
 extracting: s_9_feature.mat         
 extracting: s_3_feature.mat         
 extracting: s_8_feature.mat         
 extracting: s_10_feature.mat        
 extracting: s_25_feature.mat        
 extracting: s_36_feature.mat        
 extracting: s_47_feature.mat        
 extracting: s_71_feature.mat        
 extracting: s_63_feature.mat        
 extracting: s_69_feature.mat        
 extracting: s_83_feature.mat        
 extracting: s_20_feature.mat        
 extracting: s_70_feature.mat        
 extracting: s_51_feature.mat        
 extracting: s_60_feature.mat        
 extracting: s_68_feature.mat        
 extracting: s_92_feature.mat        
 extracting: s_99_feature.mat    

In [7]:
!mkdir /content

In [9]:
cd /content

/content


In [1]:
ls

 Cluster_index_mat/          Dataset_shuai_zip.zip   toy_data/
'Data_cleaning file.ipynb'   Features/               train_config.py
 Dataset_shuai/              README.md               train.py
 Dataset_shuai_zip/          torch


In [1]:
!ls

Dataset_shuai_zip  Dataset_shuai_zip.zip  sample_data


In [25]:
!git clone https://github.com/naimurborno/Shuai-Huang-files.git

Cloning into 'Shuai-Huang-files'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 37 (delta 13), reused 28 (delta 7), pack-reused 0 (from 0)
Receiving objects: 100% (37/37), 109.26 KiB | 1.42 MiB/s, done.
Resolving deltas: 100% (13/13), done.


In [24]:
!rm -r Shuai-Huang-files/

In [16]:
!rm -r Shuai-Huang-files/

In [27]:
cd Shuai-Huang-files/

/content/Shuai-Huang-files


In [28]:
ls

'Data_cleaning file.ipynb'   README.md   train_config.py
 dataloader.py               torch       train.py


In [29]:
!python train.py

Dataset split → Train: 350 | Val: 75 | Test: 75
/content/Dataset_shuai_zip/label.mat
/content/Dataset_shuai_zip/label.mat
/content/Dataset_shuai_zip/label.mat
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)

Batch shapes:
features    : torch.Size([8, 400, 1632])
labels      : torch.Size([8])
subject_ids : tensor([ 93,  90, 136, 396, 327, 482,  38, 446])


In [9]:
ls

'Data_cleaning file.ipynb'   __pycache__/   torch             train.py
 dataloader.py               README.md      train_config.py


In [9]:
!pip install transformers

In [1]:
!lscpu

Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          39 bits physical, 48 bits virtual
  Byte Order:             Little Endian
CPU(s):                   8
  On-line CPU(s) list:    0-7
Vendor ID:                GenuineIntel
  Model name:             Intel(R) Core(TM) i5-8265U CPU @ 1.60GHz
    CPU family:           6
    Model:                142
    Thread(s) per core:   2
    Core(s) per socket:   4
    Socket(s):            1
    Stepping:             11
    CPU(s) scaling MHz:   76%
    CPU max MHz:          3900.0000
    CPU min MHz:          400.0000
    BogoMIPS:             3600.00
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge m
                          ca cmov pat pse36 clflush dts acpi mmx fxsr sse sse2 s
                          s ht tm pbe syscall nx pdpe1gb rdtscp lm constant_tsc 
                          art arch_perfmon pebs bts rep_good nopl xtopology nons
                          top_ts

In [3]:
import numpy as np
cluster=np.load("Cluster_index_mat/s_1_cluster_index.npy",allow_pickle=True)
Feature=np.load("/media/borno/Downloads/Downloads/SHUAI HUANG PROJECT/Features/s_1_feature.npy",allow_pickle=True)

In [50]:
H, W, D=cluster.item()['cluster_index_mat'].shape
D_feat=Feature.item()['feature_mat'].shape[1]
print(H,W,D,D_feat)
Q=np.zeros((H,W,D,D_feat),dtype=np.float32)
for roi_id in range(1, Feature.item()['feature_mat'].shape[0]+1):
    mask=(cluster.item()['cluster_index_mat']==roi_id)
    Q[mask]=Feature.item()['feature_mat'][roi_id-1]

45 54 45 1632


In [52]:
Q

array([[[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
    

In [11]:
Feature.item()['feature_mat']

array([[ 2.60000000e+01,  1.40000000e+01,  2.40000000e+01, ...,
         5.11272531e-01,  8.74862871e-03,  1.59172456e-01],
       [ 1.80000000e+01,  3.40000000e+01,  2.00000000e+01, ...,
        -5.46803403e-02,  2.00385133e-02,  2.34048409e-02],
       [ 2.40000000e+01,  2.20000000e+01,  1.20000000e+01, ...,
         8.77231914e-02, -3.14283801e-01, -1.54941816e-01],
       ...,
       [ 3.10000000e+01,  2.90000000e+01,  2.90000000e+01, ...,
         4.98049941e-01,  1.24301529e-01,  2.98289081e-01],
       [ 1.70000000e+01,  2.90000000e+01,  3.20000000e+01, ...,
        -8.00683281e-03,  3.18871218e-01, -1.80601885e-01],
       [ 2.10000000e+01,  2.30000000e+01,  1.60000000e+01, ...,
        -1.88375153e-01, -5.05291048e-02,  1.59986802e-01]])

yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
